Question 1

**(Pricing a forward contract with dividends)**

The current price of silver is `$206` per ounce. The storage cost is `$1` ounce per year, payable quarterly in advance. Assuming a constant annual interest rate of $9%$ compounded quarterly, what is the theoretical forward price of silver for delivery in $9$ months?

Please submit your answer rounded to the nearest integer - for example, if your answer is 19.8, round it to either 20.

In [1]:
import math

# Given parameters
S = 206  # Current spot price of silver
r = 0.09  # Annual interest rate
t = 9/12  # Time to delivery in years
q = 4  # Number of quarters in a year
storage_cost = 1  # Annual storage cost per ounce

# Calculate the total storage cost for 9 months
total_storage_cost = storage_cost * (3/4)

# Calculate the forward price
forward_price = (S + total_storage_cost) * math.exp(r * t)

print(f"The theoretical forward price of silver for delivery in 9 months is ${forward_price:.2f} per ounce")

The theoretical forward price of silver for delivery in 9 months is $221.19 per ounce


Question 2

**(Pricing Call Options)**

Consider a 1-period binomial model with
$R = 1.05$, $S_0=50$, $u=1/d=1.08$. What is the value of a European call option on the stock with strike $K=52$, assuming that the stock does not pay dividends?

Please submit your answer rounded to two decimal places. So for example, if your answer is 5.489 then you should submit an answer of 5.48 or 5.49.

In [2]:
import math

# Given parameters
R = 1.05  # Risk-free rate
S0 = 50   # Initial stock price
u = 1.08  # Up factor
d = 1/u   # Down factor
K = 52    # Strike price

# Calculate stock prices at t=1
Su = S0 * u  # Stock price in up state
Sd = S0 * d  # Stock price in down state

# Calculate option payoffs at t=1
Cu = max(Su - K, 0)  # Call payoff in up state
Cd = max(Sd - K, 0)  # Call payoff in down state

# Calculate risk-neutral probability
p = (R - d) / (u - d)

# Calculate option price at t=0
C0 = (p * Cu + (1-p) * Cd) / R

print(f"The value of the European call option is ${C0:.2f}")

The value of the European call option is $1.53


Question 3

**(Minimum-variance Hedging)**

A farmer has a crop of grapefruit juice that will be ready for harvest and sale as $150000$ pounds of grapefruit juice in $3% months. He is worried about possible price changes, so he is considering hedging - a financial engineering technique that minimizes future uncertainties in the cash flow. Typically, hedging is carried out using futures contract. However, unfortunately, there is no 3-month futures contract for grapefruit juice, but there is a futures contract for orange juice. Still, the farmer might consider using the 3-month futures contract for orange juice as a replacement for 3-month futures contract for grapefruit juice, in the hope that these two contracts are highly correlated due to the similarity of the underlying products.

Currently, the spot prices are `$1.20` per pound for orange juice and `$1.50` per pound for grapefruit juice. The standard deviation of the prices of orange juice and grapefruit juice is `$0.2` per year, and the correlation coefficient between their prices is about $0.7$ (highly correlated). What is the minimum variance hedge for farmer, i.e. how many orange juice futures contracts does the farmer need to purchase in total?

Hint: let $h$ be the number of orange juice futures contracts that the farmer purchases. Let $T$ be the time when the grapefruit juice will be sold (in $3$ months). The cash flow at time $T$ will be two parts: (1) profits from selling the juice (2) cash flow generated by futures contract. Can you write down the explicit form of this cash flow? Notice that this cash flow is a random variable, then we can compute $h$ by minimizing the variance of the cash flow (hence minimum-variance hedging). Here each contract size is 1 pound.

In [3]:
import numpy as np

# Given parameters
grapefruit_quantity = 150000  # pounds
time_to_maturity = 3/12  # 3 months in years
spot_orange = 1.20  # $/pound
spot_grapefruit = 1.50  # $/pound
std_dev = 0.2  # annual standard deviation for both juices
correlation = 0.7

# Calculate covariance
covariance = correlation * (std_dev ** 2)

# Calculate the minimum variance hedge ratio
hedge_ratio = covariance / (std_dev ** 2)

# Calculate the number of orange juice futures contracts
num_contracts = hedge_ratio * grapefruit_quantity

print(f"The farmer should purchase {num_contracts:.0f} orange juice futures contracts.")

The farmer should purchase 105000 orange juice futures contracts.


Question 4

**(Paradox of risk-neutral pricing)**

Consider using a binomial model to price a non-dividend paying security (e.g. a stock) where the true probability of an up-move is $p=0.5$. The initial value of the stock is $S_0=100$. Let $P_0$$ denote the time $t=0$ price of a European put option on the stock with strike $K$ that expires after some fixed periods.

Now suppose that some favorable news about the stock come about. This will cause an immediate change in the price's up-move probability (say to $p=0.7$). What do you think should happen to $P_0$ according to risk-neutral pricing?

In [4]:
import numpy as np

def binomial_euro_put(S0, K, r, T, N, u, d):
    # Calculate risk-neutral probability
    p_tilde = (np.exp(r * T/N) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = S0 * (u**np.arange(N, -1, -1)) * (d**np.arange(0, N+1, 1))

    # Calculate option payoff at maturity
    payoff = np.maximum(K - ST, 0)

    # Backward induction
    for i in range(N-1, -1, -1):
        payoff = np.exp(-r * T/N) * (p_tilde * payoff[:-1] + (1-p_tilde) * payoff[1:])

    return payoff[0]

# Parameters
S0 = 100
K = 100  # Assuming at-the-money put
r = 0.05  # Assuming 5% risk-free rate
T = 1  # 1 year to expiry
N = 100  # Number of steps
u = 1.1  # Up factor
d = 1/u  # Down factor

# Calculate put price
P0 = binomial_euro_put(S0, K, r, T, N, u, d)

print(f"The European put option price P0 is: ${P0:.2f}")

The European put option price P0 is: $33.26


This code calculates the European put option price using the binomial model with risk-neutral pricing. The actual probability p is not used in the calculation, demonstrating that changes in p do not affect the option price P<sub>0</sub>.

Question 5

**(Replicating strategy for pricing)**

To price a derivative, we can use risk-neutral pricing. As an alternative yet equivalent method, we can construct price the derivative using replicating strategies.

Consider a 1-period binomial model, where we have three nodes $N_{0,0}$, $N_{1,0}$, $N_{1,1}$. We have a cash account with fixed interest rate
$r=0.01$ - that is, if we have at $N_{0,0}$ cash account with $D$ dollars, we expect $(1+r)D$ at either $N_{1,0}$ or $N_{1,1}$. We also have a stock with price $S_{0,0}$=10 at $N_{0,0}$, $S_{1,0}$=8 at $N_{1,0}$ and $S_{1,1}$=15 at $N_{1,1}$.

Now consider an European call option at time t=1 with strike K=12. Can you price the option at $N_{0,0}$ using a replicating strategy, and what is the price?

Please submit your answer rounded to only one decimal digit - for example if your answer is 1.28, then submit 1.3. If your answer is 1.12, submit 1.1.

In [5]:
import numpy as np

# Given parameters
S00 = 10  # Initial stock price
S10 = 8   # Stock price in down state
S11 = 15  # Stock price in up state
K = 12    # Strike price
r = 0.01  # Interest rate

# Calculate option payoffs at t=1
C10 = max(S10 - K, 0)  # Payoff in down state
C11 = max(S11 - K, 0)  # Payoff in up state

# Solve for replicating portfolio
delta = (C11 - C10) / (S11 - S10)
B = (S11 * C10 - S10 * C11) / ((1 + r) * (S11 - S10))

# Calculate option price at t=0
C00 = delta * S00 + B

print(f"The price of the European call option at N0,0 is: {C00:.1f}")

The price of the European call option at N0,0 is: 0.9
